In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import matplotlib.font_manager as fm
import folium
import plotly.express as px
import json
import numpy as np
from sklearn.neighbors import BallTree
import folium
from sklearn.metrics.pairwise import haversine_distances
from sklearn.preprocessing import MinMaxScaler

In [3]:
# macOS 기본 한글 폰트 설정 (AppleSDGothicNeo 등)
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

mpl.rcParams['font.family'] = 'AppleGothic'  # Mac용 (Windows는 'Malgun Gothic')
mpl.rcParams['axes.unicode_minus'] = False  # 마이너스(-) 깨짐 방지
sns.set(font='AppleGothic', style="whitegrid")  # seaborn에도 명시적으로 폰트 적용

In [12]:
path = '/Users/yniyunj/Desktop/교육공공데이터/데이터/002-전출입생수/'

df_closed_how = pd.read_csv(path + '시도교육청_폐교목록.csv', encoding = 'utf-8')
df_schools = pd.read_csv(path + '250425-충청북도_학교별_주소_new위도_new경도.csv')

In [11]:
df_closed_how = df_closed_how[df_closed_how['급별'] == '초']
df_closed_how = df_closed_how.drop('Unnamed: 0', axis=1)
df_closed_how

,시도교육청,폐교명,폐교연도,지역교육청,급별,활용현황,주소
0,충북,소이초 충도분교,1991,음성,초,매각,음성군 소이면 충도리 704
2,충북,(구)내북초,2013,보은,초,대부,보은군 내북면 동산리 119
3,충북,(구)복대초,2023,청주,초,자체활용,청주시 흥덕구 복대동 709-4
4,충북,(구)북암초,1981,보은,초,대부,보은군 속리산면 북암리 318
5,충북,(구)옥동초,2014,진천,초,자체활용,진천군 덕산읍 옥동리 359-2
...,...,...,...,...,...,...,...
255,충북,황학초 흥덕분교,1988,영동,초,매각,영동군 상촌면 흥덕리 산7-9
256,충북,회남초 법수분교장,1991,보은,초,매각,보은군 회남면 법수리 43
257,충북,회남초 분저분교장,1992,보은,초,미활용,보은군 회남면 분저리 71
258,충북,회인초 회동분교장,1994,보은,초,매각,보은군 회인면 고석리 123-1


In [43]:
df_closed_how['활용현황'].value_counts()

활용현황
매각      133
대부       72
미활용      29
자체활용     26
Name: count, dtype: int64

In [17]:
df_schools = df_schools[df_schools['폐교여부'] == 'Y']
df_schools = df_schools.drop('Unnamed: 0', axis=1)
df_schools

,학교명,학교도로명 주소,폐교여부,new위도,new경도
8,문광초등학교덕평분교장,충청북도 괴산군 청천면 송문로덕평2길 42,Y,36.730483,127.796632
11,보광초등학교화곡분교장,충청북도 괴산군 사리면 모래재로 941,Y,36.800494,127.712900
21,추산초등학교,충청북도 괴산군 불정면 동추산길 1,Y,36.902766,127.817701
29,단천초등학교가산분교장,충청북도 단양군 단성면 선암계곡로 1092,Y,36.893309,128.305517
32,대강초등학교장정분교장,충청북도 단양군 대강면 온천로 719-7,Y,36.846632,128.395263
34,별방초등학교,충청북도 단양군 영춘면 별방창원로 449,Y,37.096357,128.415812
45,송죽초등학교,충청북도 보은군 삼승면 남부로 3927,Y,36.437957,127.734841
47,수정초등학교삼가분교장,충청북도 보은군 속리산면 삼가구병길 9,Y,36.490947,127.860049
55,미봉초등학교,충청북도 영동군 양강면 학산영동로 495,Y,36.127981,127.727205
75,대성초등학교,충청북도 옥천군 이원면 이원로 443,Y,36.212902,127.630257


In [24]:
df_closed_how[df_closed_how['폐교명'].str.contains('별방')]

,Unnamed: 0,시도교육청,폐교명,폐교연도,지역교육청,급별,활용현황,주소
100,100,충북,별방초 사지원분교장,1991,단양,초,매각,단양군 영춘면 사지원리 64-1
101,101,충북,별방초 유암분교장,1995,단양,초,대부,단양군 영춘면 유암리 253


In [22]:
import pandas as pd

# 1. 학교명 앞 3글자 컬럼 만들기
df_schools['학교명_3글자'] = df_schools['학교명'].str[:3]
df_closed_how['폐교명_3글자'] = df_closed_how['폐교명'].str[:3]

# 2. 매칭 딕셔너리 만들기: 학교명_3글자 → (폐교연도, 활용현황)
match_info = {}

for name3 in df_schools['학교명_3글자'].unique():
    matches = df_closed_how[df_closed_how['폐교명_3글자'] == name3]
    if len(matches) == 1:
        폐교연도 = matches.iloc[0]['폐교연도']
        활용현황 = matches.iloc[0]['활용현황']
        match_info[name3] = {'폐교연도': 폐교연도, '활용현황': 활용현황}

# 3. 학교별로 match_info를 참조해서 붙이기
df_schools['폐교연도'] = df_schools['학교명_3글자'].map(lambda x: match_info.get(x, {}).get('폐교연도'))
df_schools['활용현황'] = df_schools['학교명_3글자'].map(lambda x: match_info.get(x, {}).get('활용현황'))

# 4. 사용한 임시 열 삭제 (깔끔하게)
df_schools.drop(columns=['학교명_3글자'], inplace=True)
df_closed_how.drop(columns=['폐교명_3글자'], inplace=True)

df_schools

,학교명,학교도로명 주소,폐교여부,new위도,new경도,폐교연도,활용현황
8,문광초등학교덕평분교장,충청북도 괴산군 청천면 송문로덕평2길 42,Y,36.730483,127.796632,2011.0,매각
11,보광초등학교화곡분교장,충청북도 괴산군 사리면 모래재로 941,Y,36.800494,127.712900,NaN,None
21,추산초등학교,충청북도 괴산군 불정면 동추산길 1,Y,36.902766,127.817701,NaN,None
29,단천초등학교가산분교장,충청북도 단양군 단성면 선암계곡로 1092,Y,36.893309,128.305517,NaN,None
32,대강초등학교장정분교장,충청북도 단양군 대강면 온천로 719-7,Y,36.846632,128.395263,NaN,None
34,별방초등학교,충청북도 단양군 영춘면 별방창원로 449,Y,37.096357,128.415812,NaN,None
45,송죽초등학교,충청북도 보은군 삼승면 남부로 3927,Y,36.437957,127.734841,NaN,None
47,수정초등학교삼가분교장,충청북도 보은군 속리산면 삼가구병길 9,Y,36.490947,127.860049,NaN,None
55,미봉초등학교,충청북도 영동군 양강면 학산영동로 495,Y,36.127981,127.727205,NaN,None
75,대성초등학교,충청북도 옥천군 이원면 이원로 443,Y,36.212902,127.630257,2014.0,대부


In [25]:
# 1. 폐교연도 또는 활용현황이 NaN인 학교만 뽑기
df_missing = df_schools[df_schools['폐교연도'].isna() | df_schools['활용현황'].isna()].copy()

# 2. df_closed_how에 폐교명 앞 3글자 컬럼 추가
df_closed_how['폐교명_3글자'] = df_closed_how['폐교명'].str[:3]

# 3. 하나하나 비교하면서 출력
for idx, row in df_missing.iterrows():
    school_name = row['학교명']
    school_prefix = school_name[:3]

    # 해당 3글자와 일치하는 폐교명들 찾기
    matches = df_closed_how[df_closed_how['폐교명_3글자'] == school_prefix]

    print(f"학교명: {school_name} (index: {idx})")
    if not matches.empty:
        print(matches[['폐교명', '폐교연도', '활용현황', '주소']])
    else:
        print("🔍 일치하는 폐교명 없음.")
    print('-' * 80)

학교명: 보광초등학교화곡분교장 (index: 11)
           폐교명  폐교연도 활용현황               주소
102  보광초 백마분교장  2001   매각  괴산군 사리면 중흥리 1-1
103  보광초 화곡분교장  2013   매각  괴산군 사리면 이곡리 452
--------------------------------------------------------------------------------
학교명: 추산초등학교 (index: 21)
           폐교명  폐교연도 활용현황                 주소
233        추산초  2017  미활용  괴산군 불정면 추산리 545-2
234  추산초 삼방분교장  1994   매각    괴산군 불정면 삼방리 304
--------------------------------------------------------------------------------
학교명: 단천초등학교가산분교장 (index: 29)
          폐교명  폐교연도 활용현황               주소
55  단천초 가산분교장  2019  미활용  단양군 단성면 가산리 290
56  단천초 두항분교장  1991   대부   단양군 단성면 두항리 39
--------------------------------------------------------------------------------
학교명: 대강초등학교장정분교장 (index: 32)
          폐교명  폐교연도 활용현황                주소
57  대강초 올산분교장  1992   대부   단양군 대강면 올산리 185
58  대강초 장정분교장  2017  미활용    단양군 대강면 장정리 30
59  대강초 죽령분교장  1988   매각   단양군 대강면 용부원리 82
60  대강초 황정분교장  1995   매각   단양군 대강면 황정리 194
-----------------------------------------

In [27]:
# 매칭된 정보를 딕셔너리 형태로 준비
update_info = {
    11: {'폐교연도': 2013, '활용현황': '매각'},
    21: {'폐교연도': 2017, '활용현황': '미활용'},
    29: {'폐교연도': 2019, '활용현황': '미활용'},
    32: {'폐교연도': 2017, '활용현황': '미활용'},
    47: {'폐교연도': 2018, '활용현황': '매각'},
    55: {'폐교연도': 2018, '활용현황': '대부'},
    169: {'폐교연도': 2019, '활용현황': '매각'},
    224: {'폐교연도': 2019, '활용현황': '대부'},
    228: {'폐교연도': 2013, '활용현황': '자체활용'},
    245: {'폐교연도': 2020, '활용현황': '대부'},
}

# df_schools 업데이트
for idx, info in update_info.items():
    df_schools.at[idx, '폐교연도'] = info['폐교연도']
    df_schools.at[idx, '활용현황'] = info['활용현황']

df_schools.sort_values('폐교연도')

,학교명,학교도로명 주소,폐교여부,new위도,new경도,폐교연도,활용현황
237,엄정초등학교목계분교장,충청북도 충주시 엄정면 안암목계길 11,Y,37.081351,127.890731,2009.0,대부
150,남일초등학교두산분교장,충청북도 청주시 상당구 남일면 두산길 53,Y,36.557544,127.540465,2010.0,자체활용
8,문광초등학교덕평분교장,충청북도 괴산군 청천면 송문로덕평2길 42,Y,36.730483,127.796632,2011.0,매각
171,외천초등학교,충북 청원군 부용면 부용외천2길 13-2,Y,36.540986,127.432369,2012.0,자체활용
85,남신초등학교덕생분교장,충청북도 음성군 음성읍 덕생로 495-7,Y,36.914346,127.623601,2012.0,미활용
77,이원초등학교지탄분교장,충청북도 옥천군 이원면 지탄리 734-5,Y,36.243933,127.682220,2012.0,대부
11,보광초등학교화곡분교장,충청북도 괴산군 사리면 모래재로 941,Y,36.800494,127.712900,2013.0,매각
228,노은초등학교수상분교장,충청북도 충주시 노은면 감노로 1055,Y,37.050514,127.715692,2013.0,자체활용
75,대성초등학교,충청북도 옥천군 이원면 이원로 443,Y,36.212902,127.630257,2014.0,대부
32,대강초등학교장정분교장,충청북도 단양군 대강면 온천로 719-7,Y,36.846632,128.395263,2017.0,미활용


In [28]:
# 활용현황이 NaN이면 '모름'으로 대체
df_schools['활용현황'] = df_schools['활용현황'].fillna('모름')
df_schools

,학교명,학교도로명 주소,폐교여부,new위도,new경도,폐교연도,활용현황
8,문광초등학교덕평분교장,충청북도 괴산군 청천면 송문로덕평2길 42,Y,36.730483,127.796632,2011.0,매각
11,보광초등학교화곡분교장,충청북도 괴산군 사리면 모래재로 941,Y,36.800494,127.712900,2013.0,매각
21,추산초등학교,충청북도 괴산군 불정면 동추산길 1,Y,36.902766,127.817701,2017.0,미활용
29,단천초등학교가산분교장,충청북도 단양군 단성면 선암계곡로 1092,Y,36.893309,128.305517,2019.0,미활용
32,대강초등학교장정분교장,충청북도 단양군 대강면 온천로 719-7,Y,36.846632,128.395263,2017.0,미활용
34,별방초등학교,충청북도 단양군 영춘면 별방창원로 449,Y,37.096357,128.415812,NaN,모름
45,송죽초등학교,충청북도 보은군 삼승면 남부로 3927,Y,36.437957,127.734841,NaN,모름
47,수정초등학교삼가분교장,충청북도 보은군 속리산면 삼가구병길 9,Y,36.490947,127.860049,2018.0,매각
55,미봉초등학교,충청북도 영동군 양강면 학산영동로 495,Y,36.127981,127.727205,2018.0,대부
75,대성초등학교,충청북도 옥천군 이원면 이원로 443,Y,36.212902,127.630257,2014.0,대부


In [31]:
# 활용현황별 개수 세기
df_schools['활용현황'].value_counts()

활용현황
모름      8
대부      7
미활용     5
자체활용    5
매각      4
Name: count, dtype: int64

In [36]:
!pip install xlrd==1.2.0

  Attempting uninstall: xlrd
    Found existing installation: xlrd 2.0.1
    Uninstalling xlrd-2.0.1:
      Successfully uninstalled xlrd-2.0.1


In [42]:
# pandas의 read_html 사용
df_closed_elem_all = pd.read_excel(path + '시도교육청_폐교목록.xlsx')
df_closed_elem_all[df_closed_elem_all['급별'] == '초']['활용현황'].value_counts()

/Users/yniyunj/.pyenv/versions/3.10.6/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


활용현황
매각      2491
대부       504
미활용      343
자체활용     298
Name: count, dtype: int64